In [215]:
import os
import time
from typing import List, Optional

import pandas as pd
import rich
import wandb
from rich import print
from rich.console import Console
from rich.table import Table
from tqdm.auto import tqdm

def display_pretty(df):
    # visualize using tabulate for ipython noteboooks
    from IPython.display import display

    # Assuming df is your DataFrame
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        display(
            df
        )



def login(project: Optional[str] = None, token: Optional[str] = None):
    if project is None:
        project = os.getenv("WANDB_PROJECT")
    if token is None:
        token = os.getenv("WANDB_API_KEY")
    # Set your wandb API key
    wandb.login(key=token)


def fetch_runs(
    project_list,
    exp_name_list: Optional[List[str]] = None,
    exp_term_list: Optional[List[str]] = None,
    total_runs_limit: Optional[int] = None,
):
    # Fetch runs from wandb
    api = wandb.Api()
    runs = (
        api.runs(path=project_list)
        if isinstance(project_list, str)
        else [api.runs(path=p) for p in project_list]
    )
    # runs = [run for run_list in runs for run in run_list]

    exp_name_to_summary_dict = {}
    exp_name_to_state_dict = {}
    idx = 0
    for run in tqdm(runs[0]):
        if "exp_name" not in run.config:
            continue
        exp_name = run.config["exp_name"].lower()

        if exp_name_list is not None and exp_name not in exp_name_list:
            continue

        if exp_term_list is not None:
            if not any(term in exp_name for term in exp_term_list):
                continue

        if exp_name not in exp_name_to_summary_dict:
            exp_name_to_summary_dict[exp_name] = [run.summaryMetrics]
            exp_name_to_state_dict[exp_name] = [run.state]
        else:
            exp_name_to_summary_dict[exp_name].append(run.summaryMetrics)
            exp_name_to_state_dict[exp_name].append(run.state)
        if total_runs_limit is not None and idx >= total_runs_limit:
            break
        idx += 1
    # sort the dicts by exp_name
    exp_name_to_summary_dict = dict(sorted(exp_name_to_summary_dict.items()))
    exp_name_to_state_dict = dict(sorted(exp_name_to_state_dict.items()))
    return exp_name_to_summary_dict, exp_name_to_state_dict


def fetch_run_status(exp_name_to_summary_dict, exp_name_to_state_dict):
    exp_data = {}
    exp_to_command = {
        key.lower(): value for key, value in exp_name_to_summary_dict.items()
    }
    if isinstance(exp_name_to_summary_dict, dict):
        experiments = list(exp_name_to_summary_dict.keys())
    experiments = [exp.lower() for exp in experiments]
    # Iterate through the given experiments
    with tqdm(total=len(experiments), desc="Checking experiments") as pbar:
        for exp_name in experiments:
            # Check if the experiment exists in wandb and if it has completed the testing stage
            if exp_name in exp_name_to_summary_dict:
                keys = [
                    k
                    for summary_keys in exp_name_to_summary_dict[exp_name]
                    for k in summary_keys.keys()
                ]

                testing_completed = any("testing/ensemble" in k for k in keys)
                model_compiled = any("model/num_parameters" in k for k in keys)

                currently_running = any(
                    "running" == state.lower()
                    for state in exp_name_to_state_dict[exp_name]
                )
                if "global_step" in keys:
                    current_iter = max(
                        [
                            summary_stats["global_step"]
                            for summary_stats in exp_name_to_summary_dict[
                                exp_name
                            ]
                            if "global_step" in summary_stats.keys()
                        ]
                    )
                else:
                    current_iter = 0

            else:
                testing_completed = False
                currently_running = False
                current_iter = 0

            # Append the data to the list
            exp_data[exp_name] = {
                "testing_completed": testing_completed,
                "currently_running": currently_running,
                "current_iter": current_iter,
                "model_compiled": model_compiled,
                "command": exp_to_command[exp_name],
            }
            pbar.update(1)
    return exp_data


def pretty_print_runs(exp_data):

    # Create a pandas DataFrame
    df = pd.DataFrame(
        exp_data
    ).T  # Transpose the DataFrame so that each experiment is a row

    # Create a console for rich print
    console = Console()

    # Create a table
    table = Table(show_header=True, header_style="bold magenta", style="dim")
    table.add_column("idx", justify="right")
    table.add_column("Experiment Name", width=50)
    table.add_column("Currently Running", justify="right")
    table.add_column("Testing Completed", justify="right")
    table.add_column("Current Iteration", justify="right")
    table.add_column("Model Compiled", justify="right")

    # Add rows to the table
    for idx, (exp_name, row) in enumerate(df.iterrows()):
        table.add_row(
            str(idx),
            exp_name,
            str(row["currently_running"]),
            str(row["testing_completed"]),
            str(row["current_iter"]),
            str(row["model_compiled"]),
        )

    # Print the table
    console.print(table)



In [216]:
project = "machinelearningbrewery/gate-0-9-1"
token = None


exp_name_to_summary_dict, exp_name_to_state_dict = fetch_runs(
                project_list=[project],
                exp_name_list=None,
                exp_term_list=["9032024", "10032024", "31032024", "30032024"],
                total_runs_limit=None,
            )
exp_data = fetch_run_status(
exp_name_to_summary_dict, exp_name_to_state_dict
)
            
pretty_print_runs(exp_data)

# store the dicts in json
import json

with open("exp_data.json", "w") as f:
    json.dump(exp_data, f)

# store the dicts in json
with open("exp_name_to_summary_dict.json", "w") as f:
    json.dump(exp_name_to_summary_dict, f)

# store the dicts in json
with open("exp_name_to_state_dict.json", "w") as f:
    json.dump(exp_name_to_state_dict, f)

  0%|          | 0/4811 [00:00<?, ?it/s]

In [ ]:
import json
import pandas as pd

model_names = ["ar-vit-b16", "effformer-s0", "bart", "bert", "clip-b16", "whisper", "svit-b16", "siglip-p16", "rnx50-32x4a1", "mpnet", "laion-b16", "flex-b-1200ep", "effv2-rw-s", "dino-b16", "deit3-b16", "convnextv2-base"]

model_names = list(set(model_names))
# load the json
with open("exp_data.json", "r") as f:
    exp_data = json.load(f)

# load the json
with open("exp_name_to_summary_dict.json", "r") as f:
    exp_name_to_summary_dict = json.load(f)

# load the json
with open("exp_name_to_state_dict.json", "r") as f:
    exp_name_to_state_dict = json.load(f)

In [ ]:
keys = []
for model_name, metrics in exp_name_to_summary_dict.items():
    cur_keys = [k for item in metrics for k in item.keys() if "testing" in k and "std" not in k and "episode" not in k and "background" not in k and "complete" not in k and "ensemble_3" in k and "similarities" not in k and "logits" not in k]
    
    keys.extend(cur_keys)

keys = sorted(set(keys))
# print(keys)

results = {}

for model_name in exp_name_to_summary_dict.keys():
    results[model_name] = {}
    for key in keys:
        for item in exp_name_to_summary_dict[model_name]:
            if key in item:
                dataset_name = model_name.split("-")[1] if "math" not in model_name else model_name.split("-")[1] + "-" + model_name.split("-")[2]
                results[model_name][f"{dataset_name}.{key}"] = item[key]
# delete all keys that have no value
for model_name in results.keys():
    for key in list(results[model_name].keys()):
        if results[model_name][key] is None:
            del results[model_name][key]


original_df = pd.DataFrame(results).T
# display_pretty(original_df)
df = original_df.copy()


In [ ]:
# label the first column as exp_name
df.index.name = "exp_name"
df["exp_name"] = df.index
df["experiment_series"] = df.index.str.split("-").str[0]
df["dataset_name"] = df.index.str.split("-").str[1]
# break the exp_name column by - and take the first part to be experiment series, and then check if the second part matches one of the model names in the list, and assign that as the model name
# check the exp_name column and if one of the model names matches part of the exp_name, assign it to the model_name column
# for item in df["exp_name"]:
#     for model_name in model_names:
#         print(model_name, item)
#         if model_name in item:
#             print(f"Model name: {model_name}, exp_name: {item}")
#             continue
#     print(f"Model name: None, exp_name: {item}")
df["model_name"] = df["exp_name"].apply(lambda x: next((model_name for model_name in model_names if model_name in x), None))
# set model_name to be in the second column
df = df.set_index("dataset_name", append=True)
df = df.set_index("model_name", append=True)
df = df.set_index("experiment_series", append=True)
df = df.groupby('model_name').first().reset_index()
# if a column has only None and NaNs it should be removed
df = df.dropna(axis=0, how='all')
# drop column with name winoground.testing/ensemble_3-/text_to_image_accuracy_top_5-epoch-mean
df = df.drop(columns=['winoground.testing/ensemble_3-/text_to_image_accuracy_top_5-epoch-mean'])
# drop column with name winoground.testing/ensemble_3-/image_to_text_accuracy_top_5-epoch-mean
df = df.drop(columns=['winoground.testing/ensemble_3-/image_to_text_accuracy_top_5-epoch-mean'])


# remove dataset_name column
# df = df.drop(columns=['dataset_name'])
# unify the rows by model_name


# display_pretty(df)
# pretty_display first 20 columns

In [ ]:
df = df.dropna(axis=1, how='all')

cleaned_df = df.drop(columns=[col for col in df.columns if df[col].isnull().sum() >= 14])
# remove columns with "logits" in the name
cleaned_df = cleaned_df[[col for col in cleaned_df.columns if "logits" not in col]]
cleaned_df = cleaned_df[[col for col in cleaned_df.columns if "ensemble_3--" not in col]]
cleaned_df = cleaned_df[[col for col in cleaned_df.columns if "global_step" not in col]]


# remove dataset_name column
# df = df.drop(columns=['dataset_name'])
# unify the rows by model_name

# remove columns where all values are 1.0
cleaned_df = cleaned_df[[col for col in cleaned_df.columns if cleaned_df[col].nunique() > 1]]
# replace testing/ensemble_3-/ in column names with .
cleaned_df.columns = cleaned_df.columns.str.replace("testing/ensemble_3-/", "")
cleaned_df.columns = cleaned_df.columns.str.replace("-epoch-mean", "")

# count columns
print(f"Number of columns: {len(cleaned_df.columns)}")
# display_pretty(cleaned_df)

# Count NaNs vs non-NaNs
nans = cleaned_df.isna().sum()
non_nans = cleaned_df.count()
print(f"Number of NaNs: {nans.sum()}")
print(f"Number of non-NaNs: {non_nans.sum()}")

# NaNs per row with row key being the column 1
nans_per_row = cleaned_df.isna().sum(axis=1)
print(f"Number of NaNs per row: {nans_per_row}")

Number of columns: 195

Number of NaNs: 155

Number of non-NaNs: 2965

Number of NaNs per row: 0      0
1      0
2     35
3      0
4     33
5      0
6      0
7     40
8      0
9      0
10     0
11    35
12     0
13     0
14     0
15    12
dtype: int64

In [ ]:
# replace NaNs in each column by sampling from N(mu, sigma) where mu and sigma are the mean and std of the column
import numpy as np

synthetic_df = cleaned_df.copy()
for col in synthetic_df.columns[1:-1]:
    synthetic_df[col] = synthetic_df[col].astype(float)
    mu = synthetic_df[col].mean()
    sigma = synthetic_df[col].std()
    synthetic_df[col] = synthetic_df[col].fillna(synthetic_df[col].apply(lambda x: np.random.normal(mu, sigma) if pd.isnull(x) else x))

In [ ]:
display_pretty(synthetic_df)

,model_name,acdc.dice_loss,acdc.loss,acdc.mIoU,acdc.mean_accuracy,acdc.overall_accuracy,ade20k.ce_loss,ade20k.dice_loss,ade20k.focal_loss,ade20k.loss,ade20k.mIoU,ade20k.mean_accuracy,ade20k.overall_accuracy,aircraft.accuracy_top_1,aircraft.loss,chexpert.testing/ensemble_3/0-aps,chexpert.testing/ensemble_3/0-auc,chexpert.testing/ensemble_3/0-bs,chexpert.testing/ensemble_3/1-aps,chexpert.testing/ensemble_3/1-auc,chexpert.testing/ensemble_3/1-bs,chexpert.testing/ensemble_3/2-aps,chexpert.testing/ensemble_3/2-auc,chexpert.testing/ensemble_3/2-bs,chexpert.testing/ensemble_3/3-aps,chexpert.testing/ensemble_3/3-auc,chexpert.testing/ensemble_3/3-bs,chexpert.testing/ensemble_3/4-aps,chexpert.testing/ensemble_3/4-auc,chexpert.testing/ensemble_3/4-bs,chexpert.testing/ensemble_3/aps-macro,chexpert.testing/ensemble_3/auc-macro,chexpert.testing/ensemble_3/bs-macro,chexpert.testing/ensemble_3/loss,cifar100.accuracy_top_1,cifar100.accuracy_top_5,cifar100.loss,cityscapes.ce_loss,cityscapes.dice_loss,cityscapes.focal_loss,cityscapes.loss,cityscapes.mIoU,cityscapes.mean_accuracy,cityscapes.overall_accuracy,clevr.accuracy_top_1,clevr.accuracy_top_1_colour,clevr.accuracy_top_1_count,clevr.accuracy_top_1_material,clevr.accuracy_top_1_shape,clevr.accuracy_top_1_size,clevr.accuracy_top_1_yes_no,clevr.loss,clevr.loss_colour,clevr.loss_count,clevr.loss_material,clevr.loss_shape,clevr.loss_size,clevr.loss_yes_no,clevr-math.accuracy_top_1,clevr-math.accuracy_top_5,clevr-math.loss,coco.ce_loss,coco.dice_loss,coco.focal_loss,coco.loss,coco.mIoU,coco.mean_accuracy,coco.overall_accuracy,cubirds.accuracy_top_1,cubirds.loss,diabetic.testing/ensemble_3/0-aps,diabetic.testing/ensemble_3/0-auc,diabetic.testing/ensemble_3/0-bs,diabetic.testing/ensemble_3/1-aps,diabetic.testing/ensemble_3/1-auc,diabetic.testing/ensemble_3/1-bs,diabetic.testing/ensemble_3/2-aps,diabetic.testing/ensemble_3/2-auc,diabetic.testing/ensemble_3/2-bs,diabetic.testing/ensemble_3/3-aps,diabetic.testing/ensemble_3/3-auc,diabetic.testing/ensemble_3/3-bs,diabetic.testing/ensemble_3/4-aps,diabetic.testing/ensemble_3/4-auc,diabetic.testing/ensemble_3/4-bs,diabetic.testing/ensemble_3/aps-macro,diabetic.testing/ensemble_3/auc-macro,diabetic.testing/ensemble_3/bs-macro,diabetic.testing/ensemble_3/loss,dtextures.accuracy_top_1,dtextures.loss,flickr30k.image_to_text_accuracy,flickr30k.image_to_text_accuracy_top_5,flickr30k.image_to_text_loss,flickr30k.loss,flickr30k.text_to_image_accuracy,flickr30k.text_to_image_accuracy_top_5,flickr30k.text_to_image_loss,food101.accuracy_top_1,food101.accuracy_top_5,food101.loss,fungi.accuracy_top_1,fungi.loss,ham10k.testing/ensemble_3/0-aps,ham10k.testing/ensemble_3/0-auc,ham10k.testing/ensemble_3/0-bs,ham10k.testing/ensemble_3/1-aps,ham10k.testing/ensemble_3/1-auc,ham10k.testing/ensemble_3/1-bs,ham10k.testing/ensemble_3/2-aps,ham10k.testing/ensemble_3/2-auc,ham10k.testing/ensemble_3/2-bs,ham10k.testing/ensemble_3/3-aps,ham10k.testing/ensemble_3/3-auc,ham10k.testing/ensemble_3/3-bs,ham10k.testing/ensemble_3/4-aps,ham10k.testing/ensemble_3/4-auc,ham10k.testing/ensemble_3/4-bs,ham10k.testing/ensemble_3/5-aps,ham10k.testing/ensemble_3/5-auc,ham10k.testing/ensemble_3/5-bs,ham10k.testing/ensemble_3/6-aps,ham10k.testing/ensemble_3/6-auc,ham10k.testing/ensemble_3/6-bs,ham10k.testing/ensemble_3/aps-macro,ham10k.testing/ensemble_3/auc-macro,ham10k.testing/ensemble_3/bs-macro,ham10k.testing/ensemble_3/loss,happy.accuracy_top_1,happy.accuracy_top_1_individual,happy.accuracy_top_1_species,happy.accuracy_top_5,happy.accuracy_top_5_individual,happy.accuracy_top_5_species,happy.loss,happy.loss_individual,happy.loss_species,hmdb51.accuracy_top_1,hmdb51.accuracy_top_5,hmdb51.loss,imagenet1k.accuracy_top_1,imagenet1k.accuracy_top_5,imagenet1k.loss,iwildcam.loss,iwildcam.mae_loss,iwildcam.mse_loss,kinetics.accuracy_top_1,kinetics.accuracy_top_5,kinetics.loss,mini.accuracy_top_1,mini.loss,newyorkercaptioncontest.image_to_text_accuracy,newyorkercaptioncontest.image_to_text_accuracy_top_

In [ ]:
model_df = cleaned_df.dropna()
# reset the index
model_df = model_df.reset_index()
# make every row into a sample in torch
model_df = model_df.drop(columns=["exp_name", "model_name", "index"])
# cast all columns to float
model_df = model_df.astype(float)
column_means = model_df.mean()
column_sd = model_df.std()
model_df = model_df.dropna()

display_pretty(model_df)
display_pretty(column_means)
display_pretty(column_sd)

,acdc.dice_loss,acdc.loss,acdc.mIoU,acdc.mean_accuracy,acdc.overall_accuracy,ade20k.ce_loss,ade20k.dice_loss,ade20k.focal_loss,ade20k.loss,ade20k.mIoU,ade20k.mean_accuracy,ade20k.overall_accuracy,aircraft.accuracy_top_1,aircraft.loss,chexpert.testing/ensemble_3/0-aps,chexpert.testing/ensemble_3/0-auc,chexpert.testing/ensemble_3/0-bs,chexpert.testing/ensemble_3/1-aps,chexpert.testing/ensemble_3/1-auc,chexpert.testing/ensemble_3/1-bs,chexpert.testing/ensemble_3/2-aps,chexpert.testing/ensemble_3/2-auc,chexpert.testing/ensemble_3/2-bs,chexpert.testing/ensemble_3/3-aps,chexpert.testing/ensemble_3/3-auc,chexpert.testing/ensemble_3/3-bs,chexpert.testing/ensemble_3/4-aps,chexpert.testing/ensemble_3/4-auc,chexpert.testing/ensemble_3/4-bs,chexpert.testing/ensemble_3/aps-macro,chexpert.testing/ensemble_3/auc-macro,chexpert.testing/ensemble_3/bs-macro,chexpert.testing/ensemble_3/loss,cifar100.accuracy_top_1,cifar100.accuracy_top_5,cifar100.loss,cityscapes.ce_loss,cityscapes.dice_loss,cityscapes.focal_loss,cityscapes.loss,cityscapes.mIoU,cityscapes.mean_accuracy,cityscapes.overall_accuracy,clevr.accuracy_top_1,clevr.accuracy_top_1_colour,clevr.accuracy_top_1_count,clevr.accuracy_top_1_material,clevr.accuracy_top_1_shape,clevr.accuracy_top_1_size,clevr.accuracy_top_1_yes_no,clevr.loss,clevr.loss_colour,clevr.loss_count,clevr.loss_material,clevr.loss_shape,clevr.loss_size,clevr.loss_yes_no,clevr-math.accuracy_top_1,clevr-math.accuracy_top_5,clevr-math.loss,coco.ce_loss,coco.dice_loss,coco.focal_loss,coco.loss,coco.mIoU,coco.mean_accuracy,coco.overall_accuracy,cubirds.accuracy_top_1,cubirds.loss,diabetic.testing/ensemble_3/0-aps,diabetic.testing/ensemble_3/0-auc,diabetic.testing/ensemble_3/0-bs,diabetic.testing/ensemble_3/1-aps,diabetic.testing/ensemble_3/1-auc,diabetic.testing/ensemble_3/1-bs,diabetic.testing/ensemble_3/2-aps,diabetic.testing/ensemble_3/2-auc,diabetic.testing/ensemble_3/2-bs,diabetic.testing/ensemble_3/3-aps,diabetic.testing/ensemble_3/3-auc,diabetic.testing/ensemble_3/3-bs,diabetic.testing/ensemble_3/4-aps,diabetic.testing/ensemble_3/4-auc,diabetic.testing/ensemble_3/4-bs,diabetic.testing/ensemble_3/aps-macro,diabetic.testing/ensemble_3/auc-macro,diabetic.testing/ensemble_3/bs-macro,diabetic.testing/ensemble_3/loss,dtextures.accuracy_top_1,dtextures.loss,flickr30k.image_to_text_accuracy,flickr30k.image_to_text_accuracy_top_5,flickr30k.image_to_text_loss,flickr30k.loss,flickr30k.text_to_image_accuracy,flickr30k.text_to_image_accuracy_top_5,flickr30k.text_to_image_loss,food101.accuracy_top_1,food101.accuracy_top_5,food101.loss,fungi.accuracy_top_1,fungi.loss,ham10k.testing/ensemble_3/0-aps,ham10k.testing/ensemble_3/0-auc,ham10k.testing/ensemble_3/0-bs,ham10k.testing/ensemble_3/1-aps,ham10k.testing/ensemble_3/1-auc,ham10k.testing/ensemble_3/1-bs,ham10k.testing/ensemble_3/2-aps,ham10k.testing/ensemble_3/2-auc,ham10k.testing/ensemble_3/2-bs,ham10k.testing/ensemble_3/3-aps,ham10k.testing/ensemble_3/3-auc,ham10k.testing/ensemble_3/3-bs,ham10k.testing/ensemble_3/4-aps,ham10k.testing/ensemble_3/4-auc,ham10k.testing/ensemble_3/4-bs,ham10k.testing/ensemble_3/5-aps,ham10k.testing/ensemble_3/5-auc,ham10k.testing/ensemble_3/5-bs,ham10k.testing/ensemble_3/6-aps,ham10k.testing/ensemble_3/6-auc,ham10k.testing/ensemble_3/6-bs,ham10k.testing/ensemble_3/aps-macro,ham10k.testing/ensemble_3/auc-macro,ham10k.testing/ensemble_3/bs-macro,ham10k.testing/ensemble_3/loss,happy.accuracy_top_1,happy.accuracy_top_1_individual,happy.accuracy_top_1_species,happy.accuracy_top_5,happy.accuracy_top_5_individual,happy.accuracy_top_5_species,happy.loss,happy.loss_individual,happy.loss_species,hmdb51.accuracy_top_1,hmdb51.accuracy_top_5,hmdb51.loss,imagenet1k.accuracy_top_1,imagenet1k.accuracy_top_5,imagenet1k.loss,iwildcam.loss,iwildcam.mae_loss,iwildcam.mse_loss,kinetics.accuracy_top_1,kinetics.accuracy_top_5,kinetics.loss,mini.accuracy_top_1,mini.loss,newyorkercaptioncontest.image_to_text_accuracy,newyorkercaptioncontest.image_to_text_accuracy_top_5,newyorker

acdc.dice_loss                                           0.559774
acdc.loss                                                0.562896
acdc.mIoU                                               41.505707
acdc.mean_accuracy                                      64.750026
acdc.overall_accuracy                                   64.587687
ade20k.ce_loss                                           1.733339
ade20k.dice_loss                                         0.645262
ade20k.focal_loss                                        0.361829
ade20k.loss                                              1.153950
ade20k.mIoU                                             28.975941
ade20k.mean_accuracy                                    38.724790
ade20k.overall_accuracy                                 56.661637
aircraft.accuracy_top_1                                  0.909008
aircraft.loss                                            0.450296
chexpert.testing/ensemble_3/0-aps                        0.690759
chexpert.t

acdc.dice_loss                                           0.076245
acdc.loss                                                0.077020
acdc.mIoU                                               14.442974
acdc.mean_accuracy                                      16.444012
acdc.overall_accuracy                                   16.376422
ade20k.ce_loss                                           0.867984
ade20k.dice_loss                                         0.023768
ade20k.focal_loss                                        0.214972
ade20k.loss                                              0.573179
ade20k.mIoU                                             15.680938
ade20k.mean_accuracy                                    19.773612
ade20k.overall_accuracy                                 19.243818
aircraft.accuracy_top_1                                  0.104611
aircraft.loss                                            0.382560
chexpert.testing/ensemble_3/0-aps                        0.161094
chexpert.t

In [ ]:
import torch

map_column_headers_to_index = {column: idx for idx, column in enumerate(model_df.columns)}
map_index_to_column_headers = {idx: column for idx, column in enumerate(model_df.columns)}
convert_df_to_tensor = model_df.to_numpy()
train_set = torch.tensor(convert_df_to_tensor, dtype=torch.float32)

print(train_set.shape)
print(map_column_headers_to_index)

torch.Size([10, 193])

{
    'acdc.dice_loss': 0,
    'acdc.loss': 1,
    'acdc.mIoU': 2,
    'acdc.mean_accuracy': 3,
    'acdc.overall_accuracy': 4,
    'ade20k.ce_loss': 5,
    'ade20k.dice_loss': 6,
    'ade20k.focal_loss': 7,
    'ade20k.loss': 8,
    'ade20k.mIoU': 9,
    'ade20k.mean_accuracy': 10,
    'ade20k.overall_accuracy': 11,
    'aircraft.accuracy_top_1': 12,
    'aircraft.loss': 13,
    'chexpert.testing/ensemble_3/0-aps': 14,
    'chexpert.testing/ensemble_3/0-auc': 15,
    'chexpert.testing/ensemble_3/0-bs': 16,
    'chexpert.testing/ensemble_3/1-aps': 17,
    'chexpert.testing/ensemble_3/1-auc': 18,
    'chexpert.testing/ensemble_3/1-bs': 19,
    'chexpert.testing/ensemble_3/2-aps': 20,
    'chexpert.testing/ensemble_3/2-auc': 21,
    'chexpert.testing/ensemble_3/2-bs': 22,
    'chexpert.testing/ensemble_3/3-aps': 23,
    'chexpert.testing/ensemble_3/3-auc': 24,
    'chexpert.testing/ensemble_3/3-bs': 25,
    'chexpert.testing/ensemble_3/4-aps': 26,
    'chexpert.testing/ensemble_3/4-auc': 27,
    'chexpert.testing/ensemble_3/4-bs': 28,
    'chexpert.testing/ensemble_3/aps-macro': 29,
    'chexpert.testing/ensemble_3/auc-macro': 30,
    'chexpert.testing/ensemble_3/bs-macro': 31,
    'chexpert.testing/ensemble_3/loss': 32,
    'cifar100.accuracy_top_1': 33,
    'cifar100.accuracy_top_5': 34,
    'cifar100.loss': 35,
    'cityscapes.ce_loss': 36,
    'cityscapes.dice_loss': 37,
    'cityscapes.focal_loss': 38,
    'cityscapes.loss': 39,
    'cityscapes.mIoU': 40,
    'cityscapes.mean_accuracy': 41,
    'cityscapes.overall_accuracy': 42,
    'clevr.accuracy_top_1': 43,
    'clevr.accuracy_top_1_colour': 44,
    'clevr.accuracy_top_1_count': 45,
    'clevr.accuracy_top_1_material': 46,
    'clevr.accuracy_top_1_shape': 47,
    'clevr.accuracy_top_1_size': 48,
    'clevr.accuracy_top_1_yes_no': 49,
    'clevr.loss': 50,
    'clevr.loss_colour': 51,
    'clevr.loss_count': 52,
    'clevr.loss_material': 53,
    'clevr.loss_shape': 54,
    'clevr.loss_size': 55,
    'clevr.loss_yes_no': 56,
    'clevr-math.accuracy_top_1': 57,
    'clevr-math.accuracy_top_5': 58,
    'clevr-math.loss': 59,
    'coco.ce_loss': 60,
    'coco.dice_loss': 61,
    'coco.focal_loss': 62,
    'coco.loss': 63,
    'coco.mIoU': 64,
    'coco.mean_accuracy': 65,
    'coco.overall_accuracy': 66,
    'cubirds.accuracy_top_1': 67,
    'cubirds.loss': 68,
    'diabetic.testing/ensemble_3/0-aps': 69,
    'diabetic.testing/ensemble_3/0-auc': 70,
    'diabetic.testing/ensemble_3/0-bs': 71,
    'diabetic.testing/ensemble_3/1-aps': 72,
    'diabetic.testing/ensemble_3/1-auc': 73,
    'diabetic.testing/ensemble_3/1-bs': 74,
    'diabetic.testing/ensemble_3/2-aps': 75,
    'diabetic.testing/ensemble_3/2-auc': 76,
    'diabetic.testing/ensemble_3/2-bs': 77,
    'diabetic.testing/ensemble_3/3-aps': 78,
    'diabetic.testing/ensemble_3/3-auc': 79,
    'diabetic.testing/ensemble_3/3-bs': 80,
    'diabetic.testing/ensemble_3/4-aps': 81,
    'diabetic.testing/ensemble_3/4-auc': 82,
    'diabetic.testing/ensemble_3/4-bs': 83,
    'diabetic.testing/ensemble_3/aps-macro': 84,
    'diabetic.testing/ensemble_3/auc-macro': 85,
    'diabetic.testing/ensemble_3/bs-macro': 86,
    'diabetic.testing/ensemble_3/loss': 87,
    'dtextures.accuracy_top_1': 88,
    'dtextures.loss': 89,
    'flickr30k.image_to_text_accuracy': 90,
    'flickr30k.image_to_text_accuracy_top_5': 91,
    'flickr30k.image_to_text_loss': 92,
    'flickr30k.loss': 93,
    'flickr30k.text_to_image_accuracy': 94,
    'flickr30k.text_to_image_accuracy_top_5': 95,
    'flickr30k.text_to_image_loss': 96,
    'food101.accuracy_top_1': 97,
    'food101.accuracy_top_5': 98,
    'food101.loss': 99,
    'fungi.accuracy_top_1': 100,
    'fungi.loss': 101,
    'ham10k.testing/ensemble_3/0-aps': 102,
    'ham10k.testing/ensemble_3/0-auc': 103,
    'ham10k.testing/ensemble_3/0-bs': 104,
    'ham10k.testing/ensemble_3/1-aps': 105,
    'ham10k.testing/ensemble_3/1-auc': 106,
    'ham10k.testing/ensemble_3/1-bs': 107,
    'ham10k.testi

In [ ]:
rows_with_nans = cleaned_df[cleaned_df.isnull().any(axis=1)]
# store the headers of the columns for a given row that are not NaNs, there should be one list for each column
nan_cell_boolean = rows_with_nans.isnull()
# store the headers of the columns for a given row that are NaNs
NaN_metric_per_row = nan_cell_boolean.apply(lambda x: list(x.index[x]), axis=1).to_list()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, hidden_size)
        self.l4 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out = self.l1(x)
        out = F.leaky_relu(out)
        # out = F.dropout(out, p=0.8)
        out = self.l2(out)
        out = F.leaky_relu(out)
        # out = F.dropout(out, p=0.8)
        out = self.l3(out)
        out = F.leaky_relu(out)
        out = self.l4(out)
        out = F.tanh(out) * 10
        
        return out

In [ ]:
import random
from sklearn.model_selection import ShuffleSplit


def train_eval_neural_network(train_df, target_columns, test_df, num_epochs, num_hidden_units):
    
    all_columns = train_df.columns
    train_tensor = torch.tensor(train_df.to_numpy(), dtype=torch.float32)
    # normalize column-wise
    full_mean = train_tensor.mean(axis=0)
    full_sd = train_tensor.std(axis=0)
    
    train_column_names = [col for col in all_columns if col not in target_columns]
    x_train = train_tensor[:, [map_column_headers_to_index[col] for col in train_column_names]]
    input_mean = full_mean[[map_column_headers_to_index[col] for col in train_column_names]]
    input_sd = full_sd[[map_column_headers_to_index[col] for col in train_column_names]]
    x_train = (x_train - input_mean) / input_sd
    
    target_columns = [col for col in all_columns if col not in train_column_names]
    y_train = train_tensor[:, [map_column_headers_to_index[col] for col in target_columns]]
    target_mean = full_mean[[map_column_headers_to_index[col] for col in target_columns]]
    target_sd = full_sd[[map_column_headers_to_index[col] for col in target_columns]]
    y_train = (y_train - target_mean) / target_sd
    # remove NaN from Series test_df and convert it to tensor
    clean_test_df: pd.Series = test_df.dropna()[1:-1].astype(float).values
    # Convert to numpy
    x_test = torch.tensor(clean_test_df, dtype=torch.float32)
    x_test = x_test - input_mean / input_sd
    
    result_dict = {key: [] for key in target_columns}
    
    kfold = ShuffleSplit(n_splits=50, test_size=0.2, random_state=42)
    average_train_loss = []
    average_val_loss = []
    
    for train_ids, val_ids in kfold.split(x_train):
        x_train_fold = x_train[train_ids]
        y_train_fold = y_train[train_ids]
        x_val_fold = x_train[val_ids]
        y_val_fold = y_train[val_ids]
        model = NeuralNet(x_train_fold.shape[1], num_hidden_units, y_train_fold.shape[1])
        optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.01)
        # print("------------------------------------------------------------")
        for _ in range(num_epochs):
            model.train()
            optimizer.zero_grad()
            y_pred = model(x_train_fold)
            loss = F.mse_loss(y_pred, y_train_fold) + F.l1_loss(y_pred, y_train_fold)
            loss.backward()
            optimizer.step()
            model.eval()
            pred_val = model(x_val_fold)
            val_loss = F.mse_loss(pred_val, y_val_fold) + F.l1_loss(pred_val, y_val_fold)
            
            
        average_train_loss.append(loss.item())
        average_val_loss.append(val_loss.item())
        y_pred = model.forward(x_test)
        y_pred_rec = y_pred * target_sd + target_mean
        updated_dict = {key: y_pred_rec[idx].item() for idx, key in enumerate(target_columns)}
        result_dict = {key: result_dict[key] + [updated_dict[key]] for key in target_columns}
    
    print(f"Mean train loss: {torch.tensor(average_train_loss).mean().item()}")
    print(f"Mean val loss: {torch.tensor(average_val_loss).mean().item()}")
    return result_dict
        
    

In [ ]:
first_row_with_nans = rows_with_nans.iloc[0]
for i in range(1, 11):
    test_output_dict = train_eval_neural_network(train_df=model_df, 
                                                 target_columns=NaN_metric_per_row[0], 
                                                 test_df=first_row_with_nans, 
                                                 num_epochs=200, 
                                                 num_hidden_units=2**i)

Mean train loss: 0.9022672772407532

Mean val loss: 4.083615779876709

Mean train loss: 0.47149890661239624

Mean val loss: 4.717853546142578

Mean train loss: 0.19106310606002808

...

Mean val loss: 8.671116828918457

Mean train loss: 3.4326963424682617

Mean val loss: 31.673490524291992

Mean train loss: 72.43013763427734

Mean val loss: 89.41228485107422

In [ ]:
print(test_output_dict)

{
    'ade20k.ce_loss': [
        7.41309928894043,
        -4.347420692443848,
        -4.347420692443848,
        -4.347420692443848,
        7.41309928894043,
        7.41309928894043,
        7.41309928894043,
        -4.347420692443848,
        7.41309928894043,
        -4.347420692443848,
        7.41309928894043,
        -4.347420692443848,
        -4.347420692443848,
        7.41309928894043,
        -4.347420692443848,
        7.41309928894043,
        -4.347420692443848,
        7.41309928894043,
        7.41309928894043,
        7.41309928894043,
        -4.347420692443848,
        -4.347420692443848,
        -4.347420692443848,
        7.41309928894043,
        -4.347420692443848,
        7.41309928894043,
        7.41309928894043,
        7.41309928894043,
        -4.347420692443848,
        -4.347420692443848,
        7.41309928894043,
        7.41309928894043,
        -4.347420692443848,
        -4.347420692443848,
        -4.347420692443848,
        -4.347420692443848,
        7.41309928894043,
        7.41309928894043,
        7.41309928894043,
        -4.347420692443848,
        7.41309928894043,
        7.41309928894043,
        -4.347420692443848,
        -4.347420692443848,
        7.41309928894043,
        -4.347420692443848,
        7.41309928894043,
        7.41309928894043,
        7.41309928894043,
        -4.347420692443848
    ],
    'ade20k.dice_loss': [
        0.8731518983840942,
        0.8731518983840942,
        0.8731518983840942,
        0.8731518983840942,
        0.8731518983840942,
        0.41178131103515625,
        0.8731518983840942,
        0.8731518983840942,
        0.8731518983840942,
        0.41178131103515625,
        0.41178131103515625,
        0.41178131103515625,
        0.41178131103515625,
        0.8731518983840942,
        0.8731518983840942,
        0.8731518983840942,
        0.41178131103515625,
        0.41178131103515625,
        0.8731518983840942,
        0.8731518983840942,
        0.41178131103515625,
        0.41178131103515625,
        0.8731518983840942,
        0.41178131103515625,
        0.8731518983840942,
        0.41178131103515625,
        0.8731518983840942,
        0.8731518983840942,
        0.41178131103515625,
        0.41178131103515625,
        0.8731518983840942,
        0.41178131103515625,
        0.8731518983840942,
        0.41178131103515625,
        0.8731518983840942,
        0.8731518983840942,
        0.8731518983840942,
        0.8731518983840942,
        0.8731518983840942,
        0.8731518983840942,
        0.41178131103515625,
        0.8731518983840942,
        0.8731518983840942,
        0.41178131103515625,
        0.41178131103515625,
        0.41178131103515625,
        0.41178131103515625,
        0.8731518983840942,
        0.8731518983840942,
        0.8731518983840942
    ],
    'ade20k.focal_loss': [
        1.6952232122421265,
        1.6952232122421265,
        1.6952232122421265,
        -1.0741780996322632,
        1.6952232122421265,
        -1.0741780996322632,
        1.6952232122421265,
        -1.0741780996322632,
        -1.0741780996322632,
        1.6952232122421265,
        1.6952232122421265,
        -1.0741780996322632,
        -1.0741780996322632,
        1.6952232122421265,
        -1.0741780996322632,
        1.6952232122421265,
        -1.0741780996322632,
        -1.0741780996322632,
        1.6952232122421265,
        1.6952232122421265,
        1.6952232122421265,
        -1.0741780996322632,
        1.6952232122421265,
        -1.0741780996322632,
        1.6952232122421265,
        -1.0741780996322632,
        -1.0741780996322632,
        1.6952232122421265,
        -1.0741780996322632,
        -1.0741780996322632,
        1.6952232122421265,
        1.6952232122421265,
        1.6952232122421265,
        1.6952232122421265,
        1.6952232122421265,
        -1.0741780996322632,
        1.6952232122421265,
        -1.0741780996322632,
        -1.0741780996322632,
        1.6952232122421265,
   